In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv('final_file.csv',encoding="GBK").drop(['Unnamed: 0'], axis=1)
X = df.values[:, 0:216]
Y = df.values[:,216]

from sklearn.cross_validation import train_test_split

In [3]:
df.head()

,shop_id,date.day_1,date.day_2,date.day_3,date.day_4,date.day_5,date.day_6,date.day_7,date.day_8,date.day_9,...,cate_2_name_快餐,cate_2_name_本地购物,cate_2_name_汤/粥/煲/砂锅/炖菜,cate_2_name_火锅,cate_2_name_烘焙糕点,cate_2_name_烧烤,cate_2_name_网吧网咖,cate_2_name_药店,cate_2_name_超市,count
0,544,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,221
1,810,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,125
2,163,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,91
3,1977,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,137
4,746,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,68


In [4]:
from sklearn import datasets, linear_model, cross_validation, grid_search
from sklearn.metrics import r2_score 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [5]:
df.shape

(555315, 217)

In [54]:
555315 - len(df[df['shop_id'] == 1629])

554990

In [56]:
554623 - len(df[df['shop_id'] == 1821])

554298

In [6]:
df1 = df[df['shop_id'] != 1629]

In [7]:
df1 = df1[df1['shop_id'] != 58]

In [8]:
df1 = df1[df1['shop_id'] != 1821]

In [9]:
df1.shape

(554298, 217)

In [10]:
X = df1.values[:, 0:216]
Y = df1.values[:,216]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

In [28]:
sum(X_train[:,0]==58)

0

In [14]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train):
    x_tr, x_cv = X_train[train_index], X_train[test_index]
    y_tr, y_cv = y_train[train_index], y_train[test_index]
    T_train_xgb = xgb.DMatrix(x_tr, y_tr)
    #params_3 = {"booster":"gbtree", "objective": "reg:linear", "max_depth":50,"eval_metric":"rmse"}
    params_3 = {"booster":"gbtree", "objective": "reg:linear", "max_depth":50,"eval_metric":"rmse",
                "subsample":0.6,"colsample_bytree":0.75}
    gbm_p_3 = xgb.train(dtrain=T_train_xgb,params=params_3)
    y_pr = gbm_p_3.predict(xgb.DMatrix(x_cv))
    r2_results.append(r2_score(y_cv, y_pr))
    mse_results.append(mean_squared_error(y_cv, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)

average_r2 0.836784458652
average_mse 2221.14049357
median_r2 0.836813961528
median_mse 2156.14360514


In [16]:
y_pred = gbm_p_3.predict(xgb.DMatrix(X_test))
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.840878572595
2258.92378068


In [36]:
X_final_1 = pd.read_csv('predictions.csv', encoding="GBK").drop(['Unnamed: 0'], axis=1)
X_final_1.head()

,shop_id,date.day_1,date.day_2,date.day_3,date.day_4,date.day_5,date.day_6,date.day_7,date.day_8,date.day_9,...,cate_2_name_小吃,cate_2_name_快餐,cate_2_name_本地购物,cate_2_name_汤/粥/煲/砂锅/炖菜,cate_2_name_火锅,cate_2_name_烘焙糕点,cate_2_name_烧烤,cate_2_name_网吧网咖,cate_2_name_药店,cate_2_name_超市
0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
X_final_1 = X_final_1[X_final_1['shop_id'] != 1821]
X_final_1 = X_final_1[X_final_1['shop_id'] != 58]
X_final_1 = X_final_1[X_final_1['shop_id'] != 1629]


In [38]:
X_final_1.shape

(25914, 216)

In [39]:
y_final = gbm_p_3.predict(xgb.DMatrix(X_final_1.values))

In [40]:
X_final_1['pred'] = y_final

In [41]:
X_f = X_final_1[['shop_id', 'pred']]

In [42]:
X_f.head()

,shop_id,pred
0,1,227.440460
1,2,74.444511
2,3,35.556892
3,4,79.108948
4,5,148.998062


In [43]:
f = open('out.txt', 'w')
for i in range(0,2000):
    print >> f, i+1,',',np.ceil(X_f[X_f.shop_id == i+1].pred.values)  # or f.write('...\n')
f.close()

--------

In [44]:
df_1629 = df[df['shop_id'] == 1629]
df_58 = df[df['shop_id'] == 58]
df_1821 = df[df['shop_id'] == 1821]

In [47]:
df_1821

,shop_id,date.day_1,date.day_2,date.day_3,date.day_4,date.day_5,date.day_6,date.day_7,date.day_8,date.day_9,...,cate_2_name_快餐,cate_2_name_本地购物,cate_2_name_汤/粥/煲/砂锅/炖菜,cate_2_name_火锅,cate_2_name_烘焙糕点,cate_2_name_烧烤,cate_2_name_网吧网咖,cate_2_name_药店,cate_2_name_超市,count
1469,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,789
1693,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1024
2074,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,474
2777,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,613
3380,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,635
13918,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,799
14233,1821,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,644
14326,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,589
14550,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,783
14686,1821,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1100


In [81]:
X_1629 = df_1629.values[:, 1:216]
Y_1629 = df_1629.values[:,216]

X_train_1629, X_test_1629, y_train_1629, y_test_1629 = train_test_split(X_1629, Y_1629, test_size = 0.3, random_state = 100)

X_1821 = df_1821.values[:, 1:216]
Y_1821 = df_1821.values[:,216]

X_train_1821, X_test_1821, y_train_1821, y_test_1821 = train_test_split(X_1821, Y_1821, test_size = 0.3, random_state = 100)

X_58 = df_58.values[:, 1:216]
Y_58 = df_58.values[:,216]

X_train_58, X_test_58, y_train_58, y_test_58 = train_test_split(X_58, Y_58, test_size = 0.3, random_state = 100)

In [84]:
l = [1, 2, 3, 4, 5, 7, 9, 10]
e = [0.1, 0.3, 0.5]
d = [10, 20, 30, 40, 50, 60, 100]

for i in e:
    r2_results = []
    mse_results= []
    kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
    for train_index, test_index in kf_total.split(X_train_58):
        x_tr_58, x_cv_58 = X_train_58[train_index], X_train_58[test_index]
        y_tr_58, y_cv_58 = y_train_58[train_index], y_train_58[test_index]
        T_train_xgb = xgb.DMatrix(x_tr_58, y_tr_58)
        params58_3 = {"booster":"gbtree", "objective": "reg:linear", 'max_depth':40,"eval_metric":"rmse", "eta":0.3, 'lambda':4}
        gbm58_p_3 = xgb.train(dtrain=T_train_xgb,params=params58_3)
        y_pr = gbm58_p_3.predict(xgb.DMatrix(x_cv_58))
        r2_results.append(r2_score(y_cv_58, y_pr))
        mse_results.append(mean_squared_error(y_cv_58, y_pr))
    print 'average_r2', np.average(r2_results)
    print 'average_mse', np.average(mse_results)
    print 'median_r2', np.median(r2_results)
    print 'median_mse', np.median(mse_results)
    print "\n"

average_r2 -1.03205257242
average_mse 338840.081131
median_r2 -0.956453193745
median_mse 329109.719358


average_r2 0.725254295506
average_mse 47241.7385058
median_r2 0.771807815201
median_mse 31640.9026126


average_r2 0.614243754972
average_mse 58428.3394604
median_r2 0.732862127377
median_mse 45311.4213972




In [60]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train_58):
    x_tr_58, x_cv_58 = X_train_58[train_index], X_train_58[test_index]
    y_tr_58, y_cv_58 = y_train_58[train_index], y_train_58[test_index]
    T_train_xgb = xgb.DMatrix(x_tr_58, y_tr_58)
    params58_3 = {"booster":"gbtree", "objective": "reg:linear", 'max_depth':20,"eval_metric":"rmse", "eta":0.3, 'lambda':4}
    gbm58_p_3 = xgb.train(dtrain=T_train_xgb,params=params58_3)
    y_pr = gbm58_p_3.predict(xgb.DMatrix(x_cv_58))
    r2_results.append(r2_score(y_cv_58, y_pr))
    mse_results.append(mean_squared_error(y_cv_58, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)
print "\n"

average_r2 0.725254295506
average_mse 47241.7385058
median_r2 0.771807815201
median_mse 31640.9026126




In [61]:
y_pred = gbm58_p_3.predict(xgb.DMatrix(X_test_58))
print(r2_score(y_test_58, y_pred))
print(mean_squared_error(y_test_58, y_pred))

0.806873469607
25887.2294966


In [74]:
pred_58 = pd.read_csv('predictions_58.csv', encoding='GBK').drop(['Unnamed: 0'], axis=1)
pred_58 = pred_58.drop('shop_id', axis=1)

In [78]:
y_pred_58_f = gbm58_p_3.predict(xgb.DMatrix(pred_58.values))
np.ceil(y_pred_58_f)

array([ 1073.,  1120.,  1032.,  1138.,  1130.,  1221.,  1134.,  1104.,
        1141.,  1148.,  1156.,  1208.,  1131.,  1154.], dtype=float32)

In [66]:
pred_file = pd.read_csv('predictions.csv', encoding="GBK").drop(['Unnamed: 0'], axis=1)

14

In [80]:
from sklearn.svm import SVR

In [108]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train_1629):
    x_tr_1629, x_cv_1629 = X_train_1629[train_index], X_train_1629[test_index]
    y_tr_1629, y_cv_1629 = y_train_1629[train_index], y_train_1629[test_index]
    T_train_xgb = xgb.DMatrix(x_tr_1629, y_tr_1629)
    params1629_3 = {"booster":"gbtree", "objective": "reg:linear", 'max_depth':50,"eval_metric":"rmse", "eta":0.3, 
                    'lambda':1,"subsample":0.6,"colsample_bytree":0.5}
    gbm1629_p_3 = xgb.train(dtrain=T_train_xgb,params=params1629_3)
    y_pr = gbm1629_p_3.predict(xgb.DMatrix(x_cv_1629))
    r2_results.append(r2_score(y_cv_1629, y_pr))
    mse_results.append(mean_squared_error(y_cv_1629, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)
print "\n"

average_r2 0.456292644716
average_mse 223668.115706
median_r2 0.512205968223
median_mse 136977.90742




In [109]:
y_pred = gbm1629_p_3.predict(xgb.DMatrix(X_test_1629))
print(r2_score(y_test_1629, y_pred))
print(mean_squared_error(y_test_1629, y_pred))

0.584260793941
177799.188506


In [110]:
pred_1629 = pd.read_csv('predictions_1629.csv', encoding='GBK').drop(['Unnamed: 0'], axis=1)
pred_1629 = pred_1629.drop('shop_id', axis=1)
y_pred_1629_f = gbm1629_p_3.predict(xgb.DMatrix(pred_1629.values))
np.ceil(y_pred_1629_f)

array([ 1352.,  1492.,  1426.,  1430.,  2346.,  2327.,  1369.,  1105.,
        1281.,  1377.,  1411.,  2393.,  2395.,  1223.], dtype=float32)

In [135]:
r2_results = []
mse_results= []
kf_total = KFold(n_splits=20, shuffle=True, random_state=100)
for train_index, test_index in kf_total.split(X_train_1821):
    x_tr_1821, x_cv_1821 = X_train_1821[train_index], X_train_1821[test_index]
    y_tr_1821, y_cv_1821 = y_train_1821[train_index], y_train_1821[test_index]
    T_train_xgb = xgb.DMatrix(x_tr_1821, y_tr_1821)
    params1821_3 = {"booster":"gbtree", "objective": "reg:linear", 'max_depth':60,"eval_metric":"rmse", "eta":0.3, 
                    'lambda':1,"subsample":0.6,"colsample_bytree":0.75}
    gbm1821_p_3 = xgb.train(dtrain=T_train_xgb,params=params1821_3)
    y_pr = gbm1821_p_3.predict(xgb.DMatrix(x_cv_1821))
    r2_results.append(r2_score(y_cv_1821, y_pr))
    mse_results.append(mean_squared_error(y_cv_1821, y_pr))
print 'average_r2', np.average(r2_results)
print 'average_mse', np.average(mse_results)
print 'median_r2', np.median(r2_results)
print 'median_mse', np.median(mse_results)
print "\n"

average_r2 0.334078095431
average_mse 45599.9727086
median_r2 0.570870571356
median_mse 29473.7726458




In [136]:
y_pred = gbm1821_p_3.predict(xgb.DMatrix(X_test_1821))
print(r2_score(y_test_1821, y_pred))
print(mean_squared_error(y_test_1821, y_pred))

0.432291494818
49700.0500711


In [137]:
pred_1821 = pd.read_csv('predictions_1821.csv', encoding='GBK').drop(['Unnamed: 0'], axis=1)
pred_1821 = pred_1821.drop('shop_id', axis=1)
y_pred_1821_f = gbm1821_p_3.predict(xgb.DMatrix(pred_1821.values))
np.ceil(y_pred_1821_f)

array([  656.,   623.,   443.,   654.,   839.,   890.,   556.,   561.,
         565.,   434.,   635.,  1033.,   962.,   604.], dtype=float32)